In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from sklearn.ensemble import RandomForestRegressor

In [3]:
types = {'id' : 'int64',
        'item_nbr' : 'int32',  
        'store_nbr' : 'int8',
        'unit_sales' : 'float32', 
        'onpromotion' : 'object'}        

In [4]:
%%time
df_all = pd.read_csv(f'data/train.csv', parse_dates = ['date'], dtype= types, infer_datetime_format = True)

Wall time: 2min 28s


In [5]:
df_all.onpromotion.fillna(False, inplace=True)
df_all.onpromotion = df_all.onpromotion.map({'False' : False, 'True' : True})
df_all.onpromotion = df_all.onpromotion.astype(bool)

%time df_all.to_feather('tmp/raw_groceries')

Wall time: 27.4 s


In [6]:
%time df_all.describe(include='all')

Wall time: 58.7 s


,id,date,store_nbr,item_nbr,unit_sales,onpromotion
count,1.254970e+08,125497040,1.254970e+08,1.254970e+08,1.254970e+08,125497040
unique,NaN,1684,NaN,NaN,NaN,2
top,NaN,2017-07-01 00:00:00,NaN,NaN,NaN,False
freq,NaN,118194,NaN,NaN,NaN,96028767
first,NaN,2013-01-01 00:00:00,NaN,NaN,NaN,NaN
last,NaN,2017-08-15 00:00:00,NaN,NaN,NaN,NaN
mean,6.274852e+07,NaN,2.746458e+01,9.727692e+05,8.554856e+00,NaN
std,3.622788e+07,NaN,1.633051e+01,5.205336e+05,2.360515e+01,NaN
min,0.000000e+00,NaN,1.000000e+00,9.699500e+04,-1.537200e+04,NaN
25%,3.137426e+07,NaN,1.200000e+01,5.223830e+05,2.000000e+00,NaN


In [6]:
df_subset = df_all.loc[df_all['date'] > '2017-05-30 00:00:00']

In [7]:
df_test = pd.read_csv(f'data/test.csv', parse_dates = ['date'], dtype= types, infer_datetime_format = True)

df_test.onpromotion.fillna(False, inplace=True)
df_test.onpromotion = df_test.onpromotion.map({'False' : False, 'True' : True})
df_test.onpromotion = df_test.onpromotion.astype(bool)

df_test.describe(include='all')

,id,date,store_nbr,item_nbr,onpromotion
count,3.370464e+06,3370464,3.370464e+06,3.370464e+06,3370464
unique,NaN,16,NaN,NaN,2
top,NaN,2017-08-27 00:00:00,NaN,NaN,False
freq,NaN,210654,NaN,NaN,3171867
first,NaN,2017-08-16 00:00:00,NaN,NaN,NaN
last,NaN,2017-08-31 00:00:00,NaN,NaN,NaN
mean,1.271823e+08,NaN,2.750000e+01,1.244798e+06,NaN
std,9.729693e+05,NaN,1.558579e+01,5.898362e+05,NaN
min,1.254970e+08,NaN,1.000000e+00,9.699500e+04,NaN
25%,1.263397e+08,NaN,1.400000e+01,8.053210e+05,NaN


In [8]:
%time df_subset.describe(include='all')

Wall time: 3.04 s


,id,date,store_nbr,item_nbr,unit_sales,onpromotion
count,8.125670e+06,8125670,8.125670e+06,8.125670e+06,8.125670e+06,8125670
unique,NaN,77,NaN,NaN,NaN,2
top,NaN,2017-07-01 00:00:00,NaN,NaN,NaN,False
freq,NaN,118194,NaN,NaN,NaN,7144225
first,NaN,2017-05-31 00:00:00,NaN,NaN,NaN,NaN
last,NaN,2017-08-15 00:00:00,NaN,NaN,NaN,NaN
mean,1.214342e+08,NaN,2.825661e+01,1.169337e+06,8.120637e+00,NaN
std,2.345679e+06,NaN,1.631917e+01,5.866784e+05,2.479180e+01,NaN
min,1.173714e+08,NaN,1.000000e+00,9.699500e+04,-9.300000e+02,NaN
25%,1.194028e+08,NaN,1.300000e+01,6.882790e+05,2.000000e+00,NaN


In [11]:
df_subset = df_subset.reset_index()

In [34]:
df_subset = df_subset.drop(['id'], axis=1)

In [13]:
%time df_subset.to_feather('tmp/raw_groceries_subset')

Wall time: 1.19 s


In [14]:
%time add_datepart(df_subset, 'date')

Wall time: 7.44 s


In [15]:
%time df_subset.describe(include='all')

Wall time: 8.56 s


,index,store_nbr,item_nbr,unit_sales,onpromotion,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
count,8.125670e+06,8.125670e+06,8.125670e+06,8.125670e+06,8125670,8125670.0,8.125670e+06,8.125670e+06,8.125670e+06,8.125670e+06,8.125670e+06,8125670,8125670,8125670,8125670,8125670,8125670,8.125670e+06
unique,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2,1,1,NaN
top,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,NaN
freq,NaN,NaN,NaN,NaN,7144225,NaN,NaN,NaN,NaN,NaN,NaN,7807675,7792022,8020558,8007476,8125670,8125670,NaN
mean,1.214342e+08,2.825661e+01,1.169337e+06,8.120637e+00,NaN,2017.0,6.777608e+00,2.725581e+01,1.433766e+01,3.055445e+00,1.888461e+02,NaN,NaN,NaN,NaN,NaN,NaN,1.499459e+09
std,2.345679e+06,1.631917e+01,5.866784e+05,2.479180e+01,NaN,0.0,7.653865e-01,3.197347e+00,8.877465e+00,2.020915e+00,2.223350e+01,NaN,NaN,NaN,NaN,NaN,NaN,1.920974e+06
min,1.173714e+08,1.000000e+00,9.699500e+04,-9.300000e+02,NaN,2017.0,5.000000e+00,2.200000e+01,1.000000e+00,0.000000e+00,1.510000e+02,NaN,NaN,NaN,NaN,NaN,NaN,1.496189e+09
25%,1.194028e+08,1.300000e+01,6.882790e+05,2.000000e+00,NaN,2017.0,6.000000e+00,2.500000e+01,7.000000e+00,1.000000e+00,1.700000e+02,NaN,NaN,NaN,NaN,NaN,NaN,1.497830e+09
50%,1.214342e+08,2.900000e+01,1.209719e+06,4.000000e+00,NaN,2017.0,7.000000e+00,2.700000e+01,1.300000e+01,3.000000e+00,1.890000e+02,NaN,NaN,NaN,NaN,NaN,NaN,1.499472e+09
75%,1.234656e+08,4.400000e+01,1.576323e+06,8.000000e+00,NaN,2017.0,7.000000e+00,3.000000e+01,2.200000e+01,5.000000e+00,2.080000e+02,NaN,NaN,NaN,NaN,NaN,NaN,1.501114e+09


In [16]:
%time df_subset.to_feather('tmp/raw_groceries_subset')

Wall time: 3.63 s


In [36]:
df_subset.unit_sales = np.log1p(np.clip(df_subset.unit_sales, 0, None))

In [17]:
def split_vals(a, n): return a[:n].copy(), a[n:].copy()

In [37]:
n_valid = len(df_test) + 5
n_trn = len(df_subset) - n_valid
train, valid = split_vals(df_subset, n_trn)
train.shape, valid.shape

((4755201, 18), (3370469, 18))

In [38]:
def rmse(x, y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(x), y), rmse(m.predict(val), y_val),
           m.score(x, y), m.score(val, y_val)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [39]:
y = np.array(train.unit_sales, dtype=np.float32)
trn = train.drop(['unit_sales'], axis=1)
y_val = np.array(valid.unit_sales, dtype=np.float32)
val = np.array(valid.drop(['unit_sales'], axis=1), dtype=np.float32)

In [40]:
%time x = np.array(trn, dtype = np.float32)

Wall time: 7.99 s


In [41]:
m = RandomForestRegressor(n_estimators=10, min_samples_leaf=100, n_jobs=4)
%time m.fit(x, y)

Wall time: 1min 45s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=100, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=4,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [42]:
print_score(m)

[0.685248658103048, 0.7104125804652438, 0.3844985614700671, 0.32829988141192623]


In [43]:
m = RandomForestRegressor(n_estimators=10, min_samples_leaf=10, n_jobs=4)
%time m.fit(x, y)

Wall time: 2min 17s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=10, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=4,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [44]:
print_score(m)

[0.515768674354724, 0.6126435937987473, 0.6513074990036587, 0.5004603894233994]


In [45]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=10, n_jobs=4)
%time m.fit(x, y)

Wall time: 3min 50s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=10, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=4,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [46]:
print_score(m)

[0.5124773616148708, 0.609483423320542, 0.6557435739980253, 0.5056006009881231]


In [47]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=5, n_jobs=4)
%time m.fit(x, y)

Wall time: 3min 56s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=5, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=4,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [48]:
print_score(m)

[0.44323451664780394, 0.5933871081870079, 0.7424866007176831, 0.5313697151355379]


In [49]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=1, n_jobs=4)
%time m.fit(x, y)

Wall time: 5min


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=4,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [ ]:
print_score(m)